# Visualizing binding site interactions in 2D

Here's a simple example (based on an example from OpenEye) of depicting 2D interactions between a receptor and a binding site. The output is an interactive svg best visualized in a web browser or similar.

In [8]:
from openeye import oechem
from openeye import oedepict
from openeye import oegrapheme
# Example based on https://docs.eyesopen.com/toolkits/cookbook/python/_downloads/activesitemaps2img.py

# Input and output file
ifile = 'BACE1.pdb'
ofile = 'BACE1.svg'
#Width and height
width = 900
height = 600

# Set split for molecular complex
#oechem.OEConfigureSplitMolComplexOptions(itf, oechem.OESplitMolComplexSetup_LigName | oechem.OESplitMolComplexSetup_CovLig)

# Configure input and output
ifs = oechem.oemolistream()
if not ifs.open(ifile):
    raise Exception("Error: Cannot open input file.")


# Initialize protein and ligand
protein = oechem.OEGraphMol()
ligand = oechem.OEGraphMol()
    
# Read input
complexmol = oechem.OEGraphMol()
if not oechem.OEReadMolecule(ifs, complexmol):
    raise Exception("Unable to read complex from %s" % ifile)
    
# Ensure it has residues
if not oechem.OEHasResidues(complexmol):
    oechem.OEPerceiveResidues(complexmol, oechem.OEPreserveResInfo_All)
    
# Split molecular complex
water = oechem.OEGraphMol()
other = oechem.OEGraphMol()

sopts = oechem.OESplitMolComplexOptions()
pfilter = sopts.GetProteinFilter()
wfilter = sopts.GetWaterFilter()
sopts.SetProteinFilter(oechem.OEOrRoleSet(pfilter, wfilter))
filter = oechem.OEMolComplexFilterCategory_Nothing
sopts.SetWaterFilter(oechem.OEMolComplexFilterFactory(filter))

oechem.OESplitMolComplex(ligand, protein, water, other, complexmol, sopts)

# Depict image
image = oedepict.OEImage(width, height)
asite = oechem.OEInteractionHintContainer(protein, ligand)
if not asite.IsValid():
    oechem.OEThrow.Fatal("Cannot initialize active site!")
asite.SetTitle(ligand.GetTitle())

oechem.OEPerceiveInteractionHints(asite)

# depiction

oegrapheme.OEPrepareActiveSiteDepiction(asite)
oegrapheme.OERenderActiveSiteMaps(image, asite)
oedepict.OEDrawCurvedBorder(image, oedepict.OELightGreyPen, 10.0)
oedepict.OEWriteImage(ofile, image)

True